# Calculate NDVI & Extract Spectra Using Masks in Python - Tiled Data

We'll be doing this for the NIWOT site whose site id was output from last week's function. Start by importing packages, setting the working directory, downloading data, and copying and pasting the function from last week.

In [1]:
# Import required packages
import os

import pandas as pd
import earthpy as et
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Polygon
import json
import requests
import numpy as np
import h5py

# Set working directory
directory_path = os.path.join(et.io.HOME, "earth-analytics")
os.chdir(directory_path)

In [2]:
# Create path to metadata
metadata_path = os.path.join(
    "capstone", "macrosystems", "Data", "NEON_Field_Site_Metadata_20220224.csv")

# Read in metadata
metadata_df = pd.read_csv(metadata_path)

In [3]:
# Create function that will do everything above and spit out a list of site ids
def get_site_ids(data_path, bounds, column1, column_info1, column2, column_info2):
    """Open a csv file, turn it into a geopandas dataframe, and filter 
    the dataframe to only contain sites within a provided location.

    Parameters
    -----------
    data_path: string 
        A path to the csv to be opened.
    bounds: tuple
        A tuple with the order of minx, miny, maxx, maxy
    column1: pandas.Series
        A column within the dataframe that will be used to filter sites
    column_info1: string
        A string that specifies what information within column 1 you want to 
        use for filtering
    column2: pandas.Series
        A column within the dataframe that will be used to filter sites
    column_info2: string
        A string that specifies what information within column 2 you want to 
        use for filtering
     

    Returns
    -----------
    id_list : list
        A list of site ids

    """

    # Open csv file
    metadata_df = pd.read_csv(data_path)

    # Convert pandas dataframe to a geopandas dataframe
    metadata_gpd = gpd.GeoDataFrame(metadata_df,
                                geometry=gpd.points_from_xy(metadata_df.field_longitude,
                                                            metadata_df.field_latitude))

    # Assign crs to metadata_gpd
    metadata_assigned_crs = metadata_gpd.set_crs(epsg=4326)
    

    # Filter your data with a box to only using the cx method
    filter_box = box(bounds[0], bounds[1], bounds[2], bounds[3])

    # Filter
    spatially_filtered_sites = metadata_assigned_crs.cx[filter_box.bounds[0]:filter_box.bounds[2], 
                                       filter_box.bounds[1]:filter_box.bounds[3]]

    # Filter by column1
    column1_gdf = spatially_filtered_sites[(
    spatially_filtered_sites[column1].str.contains(column_info1))]
    
    # Filter by column2
    column2_gdf = column1_gdf[(
    column1_gdf[column2].str.contains(column_info2))]
    
    # Create a list of site ids (could be used in a loop for later processing)
    site_ids = column2_gdf["field_site_id"].tolist()

    return site_ids

In [4]:
# Test function
ids = get_site_ids(metadata_path, (-130, 25, -100, 50), 
                   "field_site_type", "Terrestrial", 
                   "field_dominant_nlcd_classes", "Evergreen Forest")
ids

['ABBY',
 'MOAB',
 'NIWO',
 'ONAQ',
 'RMNP',
 'SJER',
 'SOAP',
 'TEAK',
 'WREF',
 'YELL']

# Explore NIWOT Data via API

In [5]:
# Server URL
SERVER = "http://data.neonscience.org/api/v0/"

# Site Code
SITECODE = "NIWO"

# Make request, using the sites/ endpoint
site_request = requests.get(SERVER+"sites/"+SITECODE)

# Convert to Python JSON object
site_json = site_request.json()

# Check it out
site_json

{'data': {'siteCode': 'NIWO',
  'siteName': 'Niwot Ridge NEON',
  'siteDescription': 'Niwot Ridge NEON',
  'siteType': 'CORE',
  'siteLatitude': 40.05425,
  'siteLongitude': -105.58237,
  'stateCode': 'CO',
  'stateName': 'Colorado',
  'domainCode': 'D13',
  'domainName': 'Southern Rockies and Colorado Plateau',
  'deimsId': 'https://deims.org/bd3322ed-0ae8-4da5-beb5-1bd5c9205fa1',
  'releases': [{'release': 'RELEASE-2021',
    'generationDate': '2021-01-23T02:30:02Z',
    'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2021'},
   {'release': 'RELEASE-2022',
    'generationDate': '2022-01-20T17:39:46Z',
    'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2022'}],
  'dataProducts': [{'dataProductCode': 'DP1.00001.001',
    'dataProductTitle': '2D wind speed and direction',
    'availableMonths': ['2017-07',
     '2017-08',
     '2017-09',
     '2017-10',
     '2017-11',
     '2017-12',
     '2018-01',
     '2018-02',
     '2018-03',
     '2018-04',
     '2018-0

In [6]:
# Use the "keys" method to view the component of the uppermost json dictionary
site_json.keys()

dict_keys(['data'])

In [7]:
# See what is stored in "data"
site_json["data"].keys()

dict_keys(['siteCode', 'siteName', 'siteDescription', 'siteType', 'siteLatitude', 'siteLongitude', 'stateCode', 'stateName', 'domainCode', 'domainName', 'deimsId', 'releases', 'dataProducts'])

In [8]:
# Let's look at what's stored in "dataProducts"
site_json["data"]["dataProducts"]

[{'dataProductCode': 'DP1.00001.001',
  'dataProductTitle': '2D wind speed and direction',
  'availableMonths': ['2017-07',
   '2017-08',
   '2017-09',
   '2017-10',
   '2017-11',
   '2017-12',
   '2018-01',
   '2018-02',
   '2018-03',
   '2018-04',
   '2018-05',
   '2018-06',
   '2018-07',
   '2018-08',
   '2018-09',
   '2018-10',
   '2018-11',
   '2018-12',
   '2019-01',
   '2019-02',
   '2019-03',
   '2019-04',
   '2019-05',
   '2019-06',
   '2019-07',
   '2019-08',
   '2019-09',
   '2019-10',
   '2019-11',
   '2019-12',
   '2020-01',
   '2020-02',
   '2020-03',
   '2020-04',
   '2020-05',
   '2020-06',
   '2020-07',
   '2020-08',
   '2020-09',
   '2020-10',
   '2020-11',
   '2020-12',
   '2021-01',
   '2021-02',
   '2021-03',
   '2021-04',
   '2021-05',
   '2021-06',
   '2021-07',
   '2021-08',
   '2021-09',
   '2021-10',
   '2021-11',
   '2021-12',
   '2022-01',
   '2022-02',
   '2022-03'],
  'availableDataUrls': ['https://data.neonscience.org/api/v0/data/DP1.00001.001/NIWO/2017-0

In [9]:
# Print all product codes and titles in dataProducts
for product in site_json["data"]["dataProducts"]:
    print(product["dataProductCode"],product["dataProductTitle"])

DP1.00001.001 2D wind speed and direction
DP1.00002.001 Single aspirated air temperature
DP1.00003.001 Triple aspirated air temperature
DP1.00004.001 Barometric pressure
DP1.00005.001 IR biological temperature
DP1.00006.001 Precipitation
DP1.00013.001 Wet deposition chemical analysis
DP1.00014.001 Shortwave radiation (direct and diffuse pyranometer)
DP1.00017.001 Dust and particulate size distribution
DP1.00022.001 Shortwave radiation (primary pyranometer)
DP1.00023.001 Shortwave and longwave radiation (net radiometer)
DP1.00024.001 Photosynthetically active radiation (PAR)
DP1.00033.001 Phenology images
DP1.00038.001 Stable isotopes in precipitation
DP1.00040.001 Soil heat flux plate
DP1.00041.001 Soil temperature
DP1.00042.001 Snow depth and understory phenology images
DP1.00043.001 Spectral sun photometer - calibrated sky radiances
DP1.00066.001 Photosynthetically active radiation (quantum line)
DP1.00094.001 Soil water content and water salinity
DP1.00095.001 Soil CO2 concentration

In [10]:
# Let's assign a variable to a product code that we may be interested in
# We're interested in data that may contain spectral data
shortwave_rad_prodcode = "DP1.00014.001"

In [11]:
# Make a request for the data and print the data's dictionary keys
shortwave_product_request = requests.get(SERVER+"products/"+shortwave_rad_prodcode)
product_json = shortwave_product_request.json()

# Print keys for product data dictionary
print(product_json["data"].keys())

dict_keys(['productCodeLong', 'productCode', 'productCodePresentation', 'productName', 'productDescription', 'productStatus', 'productCategory', 'productHasExpanded', 'productScienceTeamAbbr', 'productScienceTeam', 'productPublicationFormatType', 'productAbstract', 'productDesignDescription', 'productStudyDescription', 'productBasicDescription', 'productExpandedDescription', 'productSensor', 'productRemarks', 'themes', 'changeLogs', 'specs', 'keywords', 'releases', 'siteCodes'])


In [12]:
# Let's look at the code, name, abstract of data product, and specs
print(product_json["data"]["productCode"])
print(product_json["data"]["productName"])
print(product_json["data"]["productAbstract"])
print(product_json["data"]["specs"])

DP1.00014.001
Shortwave radiation (direct and diffuse pyranometer)
Total, direct, and diffuse shortwave radiation, available as one- and thirty-minute averages of 1 Hz observations.  Direct radiation, also called direct beam radiation, is the solar radiation traveling in a straight line from the sun to a plane at the Earth's surface oriented perpendicular to the sun's rays. Diffuse radiation is the solar radiation scattered by particles in the atmosphere and received at a horizontal plane at the Earth's surface. Diffuse radiation comes from the entire sky dome, whereas direct radiation comes from a single direction. Total solar radiation is the sum of direct and diffuse solar radiation received at a horizontal plane at the Earth's surface.

Latency:
Data collected in any given month are published during the second full week of the following month.
[{'specId': 4782, 'specNumber': 'NEON.DOC.011081vC', 'specType': 'application/pdf', 'specSize': 516059, 'specDescription': 'NEON Algorithm T

In [13]:
# I want to look at other data's abstracts as well to find datasets that
# contain spectral data. So I am going to try to make a list of data product
# codes to loop through to get this information.

data_codes_list = ["DP1.00014.001", 
                   "DP1.00022.001", 
                   "DP1.00023.001", 
                   "DP1.00024.001", 
                   "DP1.00043.001", 
                   "DP1.00066.001", 
                   "DP1.30001.001", 
                   "DP1.30003.001", 
                   "DP1.30006.001", 
                   "DP1.30008.001", 
                   "DP1.30010.001", 
                   "DP2.30014.001", 
                   "DP2.30026.001", 
                   "DP3.30006.001", 
                   "DP3.30014.001"]

level_three_list = ["DP3.30006.001",
                    "DP3.30010.001",
                    "DP3.30011.001", 
                    "DP3.30012.001",
                    "DP3.30014.001",
                    "DP3.30015.001",
                    "DP3.30019.001",
                    "DP3.30024.001",
                    "DP3.30025.001",
                    "DP3.30026.001",
                    "DP4.00001.001",
                    "DP4.00200.001"]

for code in level_three_list:
    request = requests.get(SERVER+"products/"+code)
    product_json = request.json()
    print(product_json["data"]["productCode"])
    print(product_json["data"]["productName"])
    print(product_json["data"]["productAbstract"])
    print()

DP3.30006.001
Spectrometer orthorectified surface directional reflectance - mosaic
The NEON AOP surface directional reflectance data product is an orthorectified (UTM projection) hyperspectral raster product. It is distributed in an open HDF5 format including all 426 bands from the NEON Imaging Spectrometer. It is a calibrated and atmospherically corrected product distributed as scaled reflectance. It includes many QA and ancillary rasters used as inputs to ATCOR for atmospheric correction as well as outputs from ATCOR for diagnostic purposes. L3 reflectance is distributed in 1 km by 1 km tiles with one HDF5 file per tile including the reflectance data and all metadata and ancillary data. The mosaic is created using the most-nadir pixels from the flight lines observed with the least cloud cover. Reflectance tiles are used to to calculate Level 3 Vegetation Indices, Water Indices, LAI, and fPAR. 

Latency:
AOP data will be available 60 days after the final collection day at a site.

DP3

DP3.30026.001
Vegetation indices - spectrometer - mosaic
The Vegetation Indices data product is a collection of five spectral indices: NDVI, EVI, ARVI, PRI and SAVI. These indices use regions of reflectance spectra known to be indicators of vegetation health, vegetation health in high LAI areas, vegetation health in lush and/or humid regions, and vegetation health in mixed soil and vegetation landcover areas. The indices are derived from NEON Airborne Observation Platform (AOP) Imaging Spectrometer (NIS) data collected in North-South oriented flight lines to reduce BRDF effects. These data are processed to orthorectified directional surface reflectance and then processed to the indices. L3 Vegetation Indices are distributed in 1 km square tiles with 1 m pixels whose values are taken from the most-nadir pixel from the original flight line collections.

Latency:
AOP data will be available 60 days after the final collection day at a site.

DP4.00001.001
Summary weather statistics
The data

Judging from the descriptions above, we'll be moving forward with:

DP1.30006.001 Spectrometer orthorectified surface directional reflectance - flightline (raster data)

DP1.30008.001 Spectrometer orthrorectified at-sensor radiance - flightline (raster data)

DP3.30006.001 Spectrometer orthorectified surface directional reflectance - mosaic (raster data)

Now we'll see which dataset has what we need, which is what dataset has bands that contains the wavelengths we need (captured by this sensor: https://uavprime.com/index.php/product/rededge-mx-dual-camera-full-kit/?utm_source=Google%20Shopping&utm_campaign=Google%20Shopping%20Feed&utm_medium=cpc&utm_term=2028.

Additionally, we need the data to be available in map form, not point form, however these datasets are in raster format, so that is not an issue. If you need a refresher on basic raster concepts, visit: https://101gis.com/vector-data-vs-raster-data/.

I think that it could be any of these, I just don't understand the difference between them, honestly. Each band represents 5 nm of wavelength (https://www.neonscience.org/data-collection/imaging-spectrometer), so if there is 426 bands, the 426 x 5 nm = 2130 nm. But with the tiled tutorial (https://www.neonscience.org/resources/learning-hub/tutorials/calc-ndvi-tiles-py) they end up printing the center wavelength that corresponds to the NIR (near-infrared) and VIS (visisble) bands. The wavelength of band 58 falls within red visible light and band 90 is within the near-infrared range. But how did they know what band to select for these? It doesn't go into it. Also the center wavelength of band 90 is 829.238 nm. But the MicaSense sugguests that the NIR range is from about 810 - 865 nm. So, can any band representing a wavelength within that range be used for calculation? Band 90 was used to caluculate NDVI in the tutorial.

If I understand correctly, I need to go within the dataset I selected and find 10 bands, each one representing a wavelength in nm that does not overlap with another wavelength captured by the MicaSense. That is what I'll report to Ty that we need to use.

Let's figure out how to grab an h5 file from the dataset.

In [14]:
# Let's see when there is data available first

# Assign product code of interest to a variable
mosaic_dataset_code = "DP3.30006.001"
mosaic_request = requests.get(SERVER+"products/"+mosaic_dataset_code)
mosaic_json = mosaic_request.json()

# Print keys for product data dictionary
print(mosaic_json["data"].keys())

dict_keys(['productCodeLong', 'productCode', 'productCodePresentation', 'productName', 'productDescription', 'productStatus', 'productCategory', 'productHasExpanded', 'productScienceTeamAbbr', 'productScienceTeam', 'productPublicationFormatType', 'productAbstract', 'productDesignDescription', 'productStudyDescription', 'productBasicDescription', 'productExpandedDescription', 'productSensor', 'productRemarks', 'themes', 'changeLogs', 'specs', 'keywords', 'releases', 'siteCodes'])


In [15]:
# We're interested in the data for a specific site, so we'll take a look to
# See what's available in siteCodes

print(mosaic_json["data"]["siteCodes"][0].keys())

dict_keys(['siteCode', 'availableMonths', 'availableDataUrls', 'availableReleases'])


In [16]:
# We're interested in the available months and data urls
# View available months and corresponding API urls, then save desired URL
for site in mosaic_json["data"]["siteCodes"]:
    if(site["siteCode"] == SITECODE):
        for month in zip(site["availableMonths"],site["availableDataUrls"]): #Loop through the list of months and URLs
            print(month[0],month[1])

2017-09 https://data.neonscience.org/api/v0/data/DP3.30006.001/NIWO/2017-09
2018-08 https://data.neonscience.org/api/v0/data/DP3.30006.001/NIWO/2018-08
2019-08 https://data.neonscience.org/api/v0/data/DP3.30006.001/NIWO/2019-08
2020-08 https://data.neonscience.org/api/v0/data/DP3.30006.001/NIWO/2020-08


Now that we know when data is available, we can query some data

In [17]:
# I'm going to start with the the mosaic raster data

# Assign product code of interest to a variable
mosaic_dataset_code = "DP3.30006.001"

# Make Request
h5_request = requests.get(SERVER+"data/"+mosaic_dataset_code+"/"+SITECODE+"/"+"2020-08")
h5_json = h5_request.json()

# Look at output
h5_json

{'data': {'productCode': 'DP3.30006.001',
  'siteCode': 'NIWO',
  'month': '2020-08',
  'release': 'RELEASE-2022',
  'packages': [],
  'files': [{'name': 'NEON_D13_NIWO_DP3_447000_4433000_reflectance.h5',
    'size': 638390482,
    'md5': None,
    'crc32': None,
    'crc32c': 'eb4967e3',
    'url': 'https://storage.googleapis.com/neon-aop-products/2020/FullSite/D13/2020_NIWO_4/L3/Spectrometer/Reflectance/NEON_D13_NIWO_DP3_447000_4433000_reflectance.h5'},
   {'name': 'NEON_D13_NIWO_DP3_451000_4431000_reflectance.h5',
    'size': 572781794,
    'md5': None,
    'crc32': None,
    'crc32c': '20db9550',
    'url': 'https://storage.googleapis.com/neon-aop-products/2020/FullSite/D13/2020_NIWO_4/L3/Spectrometer/Reflectance/NEON_D13_NIWO_DP3_451000_4431000_reflectance.h5'},
   {'name': 'NEON_D13_NIWO_DP3_451000_4429000_reflectance.h5',
    'size': 577918691,
    'md5': None,
    'crc32': None,
    'crc32c': '02c96bab',
    'url': 'https://storage.googleapis.com/neon-aop-products/2020/FullSite

In [18]:
# Look at the uppermost layer of dictionary
print(h5_json.keys())

dict_keys(['data'])


In [19]:
# Look at the data keys
print(h5_json["data"].keys())

dict_keys(['productCode', 'siteCode', 'month', 'release', 'packages', 'files'])


In [20]:
# Look at what is contained in "files"
print(h5_json["data"]["files"][0].keys())

dict_keys(['name', 'size', 'md5', 'crc32', 'crc32c', 'url'])


In [21]:
# Look at file names
for file in h5_json["data"]["files"]:
    print(file["name"])

NEON_D13_NIWO_DP3_447000_4433000_reflectance.h5
NEON_D13_NIWO_DP3_451000_4431000_reflectance.h5
NEON_D13_NIWO_DP3_451000_4429000_reflectance.h5
NEON_D13_NIWO_DP3_456000_4428000_reflectance.h5
NEON_D13_NIWO_DP3_450000_4437000_reflectance.h5
NEON_D13_NIWO_DP3_446000_4433000_reflectance.h5
NEON_D13_NIWO_DP3_453000_4426000_reflectance.h5
NEON_D13_NIWO_DP3_457000_4435000_reflectance.h5
NEON_D13_NIWO_DP3_448000_4426000_reflectance.h5
NEON_D13_NIWO_DP3_454000_4428000_reflectance.h5
NEON_D13_NIWO_DP3_447000_4435000_reflectance.h5
NEON_D13_NIWO_DP3_453000_4433000_reflectance.h5
NEON_D13_NIWO_DP3_458000_4430000_reflectance.h5
NEON_D13_NIWO_DP3_445000_4434000_reflectance.h5
NEON_D13_NIWO_DP3_454000_4432000_reflectance.h5
NIWO_2020080114_L1_spectrometer_processing.pdf
NEON_D13_NIWO_DP3_458000_4435000_reflectance.h5
NEON_D13_NIWO_DP3_457000_4431000_reflectance.h5
NEON_D13_NIWO_DP3_451000_4425000_reflectance.h5
NEON_D13_NIWO_DP3_453000_4432000_reflectance.h5
NEON_D13_NIWO_DP3_458000_4427000_reflecta

In [22]:
# Get the url for the first h5 file
for file in h5_json["data"]["files"]:
    if((file["name"] == "NEON_D13_NIWO_DP3_454000_4432000_reflectance.h5")):
        print(file["url"])

https://storage.googleapis.com/neon-aop-products/2020/FullSite/D13/2020_NIWO_4/L3/Spectrometer/Reflectance/NEON_D13_NIWO_DP3_454000_4432000_reflectance.h5


In [23]:
# Read in file
# mosaic_url = "https://storage.googleapis.com/neon-aop-products/2020/FullSite/D13/2020_NIWO_4/L3/Spectrometer/Reflectance/NEON_D13_NIWO_DP3_454000_4432000_reflectance.h5"
mosaic_path = os.path.join("capstone", "macrosystems", "Data", "NEON_D13_NIWO_DP3_454000_4432000_reflectance.h5")
mosaic_data = h5py.File(mosaic_path, "r")

In [24]:
# list_dataset lists the names of datasets in an hdf5 file
def list_dataset(name,node):
    if isinstance(node, h5py.Dataset):
        print(name)

mosaic_data.visititems(list_dataset)

NIWO/Reflectance/Metadata/Ancillary_Imagery/Aerosol_Optical_Depth
NIWO/Reflectance/Metadata/Ancillary_Imagery/Aspect
NIWO/Reflectance/Metadata/Ancillary_Imagery/Cast_Shadow
NIWO/Reflectance/Metadata/Ancillary_Imagery/Dark_Dense_Vegetation_Classification
NIWO/Reflectance/Metadata/Ancillary_Imagery/Data_Selection_Index
NIWO/Reflectance/Metadata/Ancillary_Imagery/Haze_Cloud_Water_Map
NIWO/Reflectance/Metadata/Ancillary_Imagery/Illumination_Factor
NIWO/Reflectance/Metadata/Ancillary_Imagery/Path_Length
NIWO/Reflectance/Metadata/Ancillary_Imagery/Sky_View_Factor
NIWO/Reflectance/Metadata/Ancillary_Imagery/Slope
NIWO/Reflectance/Metadata/Ancillary_Imagery/Smooth_Surface_Elevation
NIWO/Reflectance/Metadata/Ancillary_Imagery/Visibility_Index_Map
NIWO/Reflectance/Metadata/Ancillary_Imagery/Water_Vapor_Column
NIWO/Reflectance/Metadata/Ancillary_Imagery/Weather_Quality_Indicator
NIWO/Reflectance/Metadata/Coordinate_System/Coordinate_System_String
NIWO/Reflectance/Metadata/Coordinate_System/EPSG C

Continue working through NEON tutorial at this point: https://www.neonscience.org/resources/learning-hub/tutorials/neon-aop-hdf5-tile-py